In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load NLP modules
import re
import sys
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.util import ngrams

In [3]:
df_business = pd.read_csv('../../data/dataset/csv/business.csv', index_col = 'business_id')
#df_checkin = pd.read_csv('../../data/dataset/csv/checkin.csv', index_col = 'business_id')
#df_photos = pd.read_csv('../../data/dataset/csv/photos.csv', index_col = 'photo_id')
df_review = pd.read_csv('../../data/dataset/csv/review.csv', index_col = 'review_id')
#df_tip = pd.read_csv('../../data/dataset/csv/tip.csv') # index not set for the df_tip
df_user = pd.read_csv('../../data/dataset/csv/user.csv', index_col = 'user_id')

In [4]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
import sklearn.utils

class PreProcessing:
    pd.options.mode.chained_assignment = None  # default='warn'
    # The constructor takes a pandas dataframe as input and save it to self.df
    def __init__(self, df_r, df_b, df_u):
        self.df_r = df_r
        self.df_b = df_b
        self.df_u = df_u
        self.token_pattern = "[a-zA-Z0-9'`]+"


    def TextPreProcessing(self, line):
        ## tokenize
        tokenizer = RegexpTokenizer(self.token_pattern)
        tokens = tokenizer.tokenize( str(line).lower() )
        return tokens

    def Myngrams(self, text, nfold):
        s = []
        for ngram in ngrams(text, nfold):
            s.append(' '.join(str(i) for i in ngram))

        #s = list(set(s)) # unique string in the list
        return s

    def TryDivide(self, x, y, val=0.0):
        """ 
        Try to divide two numbers
        """
        if y != 0.0:
            val = float(x) / y
        return val

    def GetPositionList(self, tgt, obs):
        """
        Get the list of positions of obs in target
        """
        pos_of_obs_in_tgt = [0]
        if len(obs) != 0:
            pos_of_obs_in_tgt = [j for j,w in enumerate(obs, start=1) if w in tgt]
            if len(pos_of_obs_in_tgt) == 0:
                pos_of_obs_in_tgt = [0]
        #print(pos_of_obs_in_tgt)
        return pos_of_obs_in_tgt

    def DumpTextBasicNgram(self, df):
        ## 1-gram
        print("Generate 1-gram...")
        df["text_1gram"] = list(df.apply(lambda x: self.Myngrams(self.TextPreProcessing(x["text"]), 1), axis=1))
        ## 2-gram
        print("Generate 2-gram...")
        df["text_2gram"] = list(df.apply(lambda x: self.Myngrams(self.TextPreProcessing(x["text"]), 2), axis=1))
        ## 3-gram
        print("Generate 3-gram...")
        df["text_3gram"] = list(df.apply(lambda x: self.Myngrams(self.TextPreProcessing(x["text"]), 3), axis=1))
        return df

    def DumpTextCountFeatures(self, df):
        ################################
        ## text count and digit count ##
        ################################
        print("Generate basic text count features...")
        fnames = ["text"]
        ngrams = ["1gram", "2gram", "3gram"]
        CountDigit = lambda x: sum([1. for w in x if w.isdigit()])
    
        for fname in fnames:
            for ngram in ngrams:
                ## word count
                df["Count_%s_%s"%(fname, ngram)] = list(df.apply(lambda x: len(x[fname + "_" + ngram]), axis=1))
                df["CountUnique_%s_%s"%(fname, ngram)] = list(df.apply(lambda x: len(set(x[fname + "_" + ngram])), axis=1))
                df["RatioUnique_%s_%s"%(fname, ngram)] = list(df.apply(lambda x: self.TryDivide(x["CountUnique_%s_%s"%(fname, ngram)], x["Count_%s_%s"%(fname, ngram)]), axis=1))
            ## digit count
            df["CountDigit_%s"%fname] = list(df.apply(lambda x: CountDigit(x[fname + "_1gram"]), axis=1))
            df["RatioDigit_%s"%fname] = list(df.apply(lambda x: self.TryDivide(x["CountDigit_%s"%fname], x["Count_%s_1gram"%(fname)]), axis=1))
        '''
        ## description missing indicator
        #df["description_missing"] = list(df.apply(lambda x: int(x["description_unigram"] == ""), axis=1))
        '''

        df.drop(['text_1gram', 'text_2gram', 'text_3gram'], axis=1, inplace=True)
    
        return df    
    
    def GenTrainDataset(self):
        #merge review and bussiness, based on review
        sub_df_b = self.df_b[['state', 'stars', 'review_count']]
        sub_df_b.rename(index=str, columns={"stars": "bstar_ave", "review_count": "breview_count"}, inplace=True)
        df_train_r_comb_b = pd.merge(self.df_r, sub_df_b, how='outer', left_on='business_id' , right_index=True)
    
        #merge review and user, based on review
        sub_df_u = self.df_u[['fans', 'review_count', 'friends']]
        sub_df_u.rename(index=str, columns={"review_count": "ureview_count"}, inplace=True)
        sub_df_u['nfriends'] = list(sub_df_u.apply(lambda x: len(x['friends']), axis=1))
        sub_df_u.drop(['friends'], axis=1, inplace=True)
        df_train_r_comb_b_u = pd.merge(df_train_r_comb_b, sub_df_u, how='outer', left_on='user_id', right_index=True)

        #build nlp features...
        df_train = self.DumpTextCountFeatures( self.DumpTextBasicNgram(df_train_r_comb_b_u) )
        df_train.drop(['date', 'text', 'business_id', 'user_id', 'state'], axis=1, inplace=True)
        return df_train

sub_df_review = sklearn.utils.shuffle(df_review).iloc[:50000, :]
preprocessing = PreProcessing(sub_df_review, df_business, df_user)
df_train = preprocessing.GenTrainDataset()
print ("done with train set preprocessing!")
df_train.head()

Generate 1-gram...
Generate 2-gram...


/home/huawei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: generator 'ngrams' raised StopIteration


Generate 3-gram...
Generate basic text count features...
done with train set preprocessing!


,useful,stars,cool,funny,bstar_ave,breview_count,fans,ureview_count,nfriends,Count_text_1gram,CountUnique_text_1gram,RatioUnique_text_1gram,Count_text_2gram,CountUnique_text_2gram,RatioUnique_text_2gram,Count_text_3gram,CountUnique_text_3gram,RatioUnique_text_3gram,CountDigit_text,RatioDigit_text
review_id,,,,,,,,,,,,,,,,,,,,
PiOoO6Mn3jHG8-9-Dd85PA,0.0,1.0,0.0,0.0,2.5,27.0,0.0,26.0,26.0,64,51,0.796875,63,63,1.000000,62,62,1.000000,1.0,0.015625
UsCISuucrmqwqYds-CgyIg,0.0,4.0,0.0,0.0,4.5,577.0,0.0,45.0,338.0,315,154,0.488889,314,285,0.907643,313,306,0.977636,5.0,0.015873
HitVCYoau7GqGnexA6LHLQ,2.0,2.0,0.0,0.0,2.5,98.0,0.0,45.0,338.0,273,149,0.545788,272,252,0.926471,271,266,0.981550,0.0,0.000000
JqlEkwdtaYEcEDnSA9_DPg,0.0,5.0,0.0,0.0,4.5,577.0,0.0,39.0,416.0,33,28,0.848485,32,31,0.968750,31,31,1.000000,0.0,0.000000
AoES3CCk4t20097br47UMw,0.0,5.0,0.0,0.0,4.5,577.0,2.0,106.0,3354.0,108,83,0.768519,107,107,1.000000,106,106,1.000000,0.0,0.000000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score